In [ ]:
from tqdm.auto import tqdm
import pandas as pd
import random

In [ ]:
dados = pd.read_pickle('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/Indicios/dataset_gold_hab_atualizada_chunks.pkl')
dados

In [ ]:
def retornaListaIndex(words, masked_words):
    lista_index = []
    for word in range(len(words)):
        for masked_word in range(len(masked_words)):
            if words[word] == masked_words[masked_word] and word not in lista_index:
                lista_index.append(word)
    return lista_index

In [ ]:
def mask_words(sentence: str) -> list:
    words = sentence.split()
    words_mascaraveis = [word for word in words if word.isalpha()]
    list_masked_indices = retornaListaIndex(words, words_mascaraveis)
    palavras_reais = []
    min_masks = max(1, int(len(list_masked_indices) * 0.01)) 
    max_masks = max(min_masks, int(len(list_masked_indices) * 0.15))
    num_masks = random.randint(min_masks, max_masks)
    masked_words = words.copy()
    if num_masks > 0:
        masked_indices = random.sample(list_masked_indices, num_masks)
        masked_indices = sorted(masked_indices)
        for index in masked_indices:
            palavras_reais.append(masked_words[index])
            masked_words[index] = '[MASK]'
    masked_sentence = ' '.join(masked_words)
    return masked_sentence, palavras_reais

In [ ]:
def mascaraDataFrame(df:pd.DataFrame, coluna: str) -> pd.DataFrame:
    registros = []
    for linha in tqdm(df.index, desc="Mascarando DataFrame de validação"):
        try:
            secao_limpa = df[coluna][linha]
            secao_mascarada, labels = mask_words(secao_limpa)
            registros.append({'text': secao_mascarada, 'labels': labels})
        except:
            pass
    dataframe = pd.DataFrame(registros)
    return dataframe

In [ ]:
dados = mascaraDataFrame(dados, 'text')
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/Indicios/dataset_gold_hab_atualizada_masked.csv', index=False)